In [2]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'


In [1]:
import os
import math
import warnings
warnings.filterwarnings('ignore')

from IPython.display import Image, HTML
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import pymc3 as pm


WARNING (theano.tensor.blas): Using NumPy C-API based implementation for BLAS functions.


In [2]:
season_df = pd.read_csv("1920PremierLeague")
season_df = season_df.set_index("Home \ Away")
season_df.index = season_df.columns

In [3]:
season_df.head()

,ARS,AVL,BOU,BHA,BUR,CHE,CRY,EVE,LEI,LIV,MCI,MUN,NEW,NOR,SHU,SOU,TOT,WAT,WHU,WOL
ARS,—,3–2,1–0,1–2,2–1,1–2,2–2,3–2,NaN,NaN,0–3,2–0,4–0,NaN,1–1,2–2,2–2,NaN,1–0,1–1
AVL,NaN,—,1–2,2–1,2–2,NaN,NaN,2–0,1–4,1–2,1–6,NaN,2–0,1–0,NaN,1–3,2–3,2–1,0–0,NaN
BOU,1–1,2–1,—,3–1,0–1,2–2,NaN,3–1,NaN,0–3,1–3,1–0,NaN,0–0,1–1,NaN,NaN,0–3,2–2,1–2
BHA,NaN,1–1,2–0,—,1–1,1–1,0–1,3–2,0–2,NaN,NaN,NaN,NaN,2–0,0–1,0–2,3–0,1–1,1–1,2–2
BUR,0–0,1–2,3–0,NaN,—,2–4,0–2,1–0,2–1,0–3,1–4,0–2,1–0,2–0,NaN,3–0,1–1,NaN,3–0,NaN


In [4]:
#create dataframe where each row corresponds to one game
season_df.index = season_df.columns
rows = []
for i in season_df.index:
    for c in season_df.columns:
        if i == c: continue
        score = season_df[c][i]
        if str(score) in ['nan', 'a']: continue
        score = [int(row) for row in str(score).split('–')]
        rows.append([i, c, score[0], score[1]])
df = pd.DataFrame(rows, columns = ['home', 'away', 'home_score', 'away_score'])
df.head()


,home,away,home_score,away_score
0,ARS,AVL,3,2
1,ARS,BOU,1,0
2,ARS,BHA,1,2
3,ARS,BUR,2,1
4,ARS,CHE,1,2


In [5]:
teams = season_df.columns
teams = pd.DataFrame(teams, columns=['team'])
teams['i'] = teams.index
teams.head()


,team,i
0,ARS,0
1,AVL,1
2,BOU,2
3,BHA,3
4,BUR,4


In [6]:
df = pd.merge(df, teams, left_on='home', right_on='team', how='left')
df = df.rename(columns = {'i': 'i_home'}).drop('team', 1)
df = pd.merge(df, teams, left_on='away', right_on='team', how='left')
df = df.rename(columns = {'i': 'i_away'}).drop('team', 1)
df.head()


,home,away,home_score,away_score,i_home,i_away
0,ARS,AVL,3,2,0,1
1,ARS,BOU,1,0,0,2
2,ARS,BHA,1,2,0,3
3,ARS,BUR,2,1,0,4
4,ARS,CHE,1,2,0,5


In [7]:
observed_home_goals = df.home_score.values
observed_away_goals = df.away_score.values
home_team = df.i_home.values
away_team = df.i_away.values
num_teams = len(df.i_home.unique())


In [9]:
g = df.groupby('i_away')
att_starting_points = np.log(g.away_score.mean())
g = df.groupby('i_home')
def_starting_points = -np.log(g.away_score.mean())


\begin{align}
y_{gj}|\theta_{gj} = Poisson(\theta_{gj}) \\
log(θ_{g1})= intercept + home_{h(g)} + att_{h(g)} + def_{a(g)} \\
log(θ_{g2})= intercept - home_{h(a)} + att_{a(g)} + def_{h(g)} \\
att_{t} & = Normal(0,\tau_{att}) \\
def_{t} & = Normal(0,\tau_{def}) \\
home_{t} & = Normal(0,\tau_{home}) \\
\tau_{att,def,home} & = Gamma(.1,.1)
\end{align}


### Model description
[Based off of the blog post here](http://danielweitzenfeld.github.io/passtheroc/blog/2014/10/28/bayes-premier-league/)

In each game (<sub>g</sub>) we are tring to predict the number of goals for the home team (y<sub>g1</sub>) and away team (y<sub>g2</sub>) which we sample from a poisson distribution based on the underlying attacking/defensive capabilities and home advantage of each team.

This model differs from the previous blog by making the home advantage value dependent on the team that's playing. I added this term in mostly as a way to account for Southampton performing better in away games.

In [11]:
#set our model parameters
with pm.Model() as model:
    tau_att = pm.Gamma('tau_att', alpha=.1, beta=.1)
    tau_def = pm.Gamma('tau_def', alpha=.1, beta=.1)
    tau_home = pm.Gamma('tau_home', alpha=.1, beta=.1)

    intercept = pm.Normal('intercept', mu=0, tau=.0001)

    #team specific values
    atts = pm.Normal("atts", 
                            mu=0, 
                            tau=tau_att, 
                            shape=num_teams)
    
    defs = pm.Normal("defs", 
                            mu=0, 
                            tau=tau_def, 
                            shape=num_teams) 
    
    home = pm.Normal("home", 
                            mu=0, 
                            tau=tau_home, 
                            shape=num_teams)

    
    home_theta = pm.Deterministic("home_theta", np.exp(intercept + 
                  home[home_team] + 
                  atts[home_team] + 
                  defs[away_team]))
    
    
    away_theta = pm.Deterministic("away_theta", np.exp(intercept -
              home[away_team] +
              atts[away_team] + 
              defs[home_team]))

    
    home_goals = pm.Poisson('home_goals', 
                          mu=home_theta, 
                          observed=observed_home_goals)
    
    away_goals = pm.Poisson('away_goals', 
                          mu=away_theta, 
                          observed=observed_away_goals)


In [13]:
with model:
    step = pm.Metropolis()
    trace = pm.sample(20000, tune=10000,step=step)


Multiprocess sampling (2 chains in 2 jobs)
CompoundStep
>Metropolis: [home]
>Metropolis: [defs]
>Metropolis: [atts]
>Metropolis: [intercept]
>Metropolis: [tau_home]
>Metropolis: [tau_def]
>Metropolis: [tau_att]


Sampling 2 chains for 10_000 tune and 20_000 draw iterations (20_000 + 40_000 draws total) took 170 seconds.
The estimated number of effective samples is smaller than 200 for some parameters.


Let's create a class to encapsulate some of the complexity in simulating the remaining games and creating a league table

In [111]:
class Season:
    
    def __init__(self, season_dataframe, trace):
        """
        Args:
            season_dataframe: Pandas dataframe, row names are home team, column names are away team, cell is H–A score
            trace: Dictionary containing values for ["defs","atts","home", "intercept"]
        """
        self.data = season_dataframe
        
        self.trace = trace
        
        self.teams = pd.DataFrame(season_df.columns, columns=['team'])
        self.teams['i'] = teams.index
        
    def simulate_remaining_games(self):
        result = self.data.copy()
        
        #make sure to use the same seed across games within the same season
        seed = np.random.randint(len(self.trace))
        rows = []
        for home_team in self.data.index:
            for away_team in self.data.columns:
                score = self.data[away_team][home_team]
                if str(score) in ['nan', 'a']: 
                    home_goals, away_goals = self.simulate_game(home_team,away_team,seed)
                    result[away_team][home_team] = str(home_goals) + "–" + str(away_goals)
        return result

    
    def league_table(self):
        result=pd.DataFrame(list(self.teams["team"]),columns=["Team"])
        result["Pts"]=0
        result["Gd"]=0

        #create dictionary {"ARS":int(points)}
        #turn dictionary into dataframe with position column
        for home_team in self.data.index:
            for away_team in self.data.columns:
                score = self.data[away_team][home_team]
                if str(score) in ["nan", "a", "—"]: continue
                    
                home_goals,away_goals = [int(goals) for goals in str(score).split('–')]
                result.loc[result["Team"]==home_team,"Gd"]+=home_goals-away_goals
                result.loc[result["Team"]==away_team,"Gd"]+=away_goals-home_goals

                home_points, away_points = self.goals_to_points(home_goals,away_goals)
                result.loc[result["Team"]==home_team,"Pts"]+=home_points
                result.loc[result["Team"]==away_team,"Pts"]+=away_points
                
        result = result.sort_values(["Pts","Gd"], ascending=False)
        result["Pos"]=range(1,len(result)+1)
        return result
    
    def simulate_game(self, home, away, seed=False):
        """
        Args:
            home: 3 letter name of team e.g. "ARS"
            away: 3 letter name of team e.g. "ARS"
            seed: Integer, maximum possible value is len(self.trace)
        
        Returns:
            list: [int(home_team_goals), int(away_team_goals)]
        """
        if not seed:
            seed =  np.random.randint(len(self.trace))

        home_index = self.team_index(home)
        away_index = self.team_index(away)
        
        home_attack = self.trace["atts"][seed][home_index]
        home_defence = self.trace["defs"][seed][home_index]
        away_attack = self.trace["atts"][seed][away_index]
        away_defence = self.trace["defs"][seed][away_index]
        
        home_advantage = self.trace["home"][seed][home_index]
        away_disadvantage = - self.trace["home"][seed][away_index]
        intercept = self.trace["intercept"][seed]

        home_goals = np.random.poisson(np.exp(intercept + home_advantage + home_attack + away_defence))
        away_goals = np.random.poisson(np.exp(intercept + away_disadvantage + away_attack + home_defence))
        return home_goals, away_goals
            
        
    def team_index(self, team_name):
        return int(self.teams[self.teams["team"]==team_name]["i"])
    
    
    def goals_to_points(self, home_goals, away_goals):
        """
        Args:
            home_goals: int
            away_goals: int
            
        Returns:
            list: [int(home_team_points_earned), int(away_team_points_earned)]
        """
        if home_goals == away_goals:
            return 1,1
        elif home_goals>away_goals:
            return 3,0
        else:

            return 0,3

### Run 1000 simulations to get range of positions and points total for each team
Questions to answer:
- Does Liverpool ever come in second?
- Which teams get regulated?
- Does Southampton ever make it into champions league? (i.e. make the top 5)

In [ ]:
current_season = Season(season_df,trace)
ranks = dict(zip(list(current_season.teams["team"]), [[] for i in range (20)]))
for _ in range(1000):
    simulated_season = Season(current_season.simulate_remaining_games(), trace)
    simulated_league_table = simulated_season.league_table()
    for i,row in simulated_league_table.iterrows():
        ranks[row["Team"]].append(row["Pos"])
ranks

I added home advantage values for each team, mostly as a way to take into account that Southampton plays better away than at home. So what would happen if Southampton was able to play against themselves? Would the home team win, or the away team?

In [292]:
home_wins=0
away_wins=0

for i in range(1000):
    home, away =season.simulate_game("SOU","SOU")
    if home > away:
        home_wins+=1
    elif home < away:
        away_wins+=1
    
print(home_wins,away_wins)

204 608
